In [9]:
import praw
import pandas as pd

# 用你的 Reddit API 凭证替换这里
reddit = praw.Reddit(
    client_id="D7jV133THru3cEHpEvhCKQ",
    client_secret="C5VA2Vbv51bRainAcoUf7vRMnRlqfA",
    user_agent="math-crawler by zoey"
)

subreddit_name = "learnmath"  # 可改成 math, mathmemes 等
limit = 1000  # 抓取前1000篇帖子及其评论

data = []

for submission in reddit.subreddit(subreddit_name).hot(limit=limit):
    submission.comments.replace_more(limit=0)
    for comment in submission.comments:
        data.append({
            "post_title": submission.title,
            "comment": comment.body
        })

df_reddit = pd.DataFrame(data)
df_reddit.to_csv("reddit_comments.csv", index=False)
print("Reddit 评论已保存！")

Reddit 评论已保存！


In [10]:
#✅ Step 1：加载评论并做基础清理
import pandas as pd
import re

# 读取你刚刚保存的评论数据
df = pd.read_csv("reddit_comments.csv")

# 简单文本预处理
def clean_text(text):
    text = re.sub(r"http\S+", "", text)  # 去除链接
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # 去除标点
    text = text.lower().strip()
    return text

df["cleaned_comment"] = df["comment"].astype(str).apply(clean_text)

In [11]:
#✅ Step 2：主题建模（LDA）
import gensim
from gensim import corpora
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')

# 分词 + 去停用词
stop_words = set(stopwords.words('english'))
df["tokens"] = df["cleaned_comment"].apply(lambda x: [word for word in word_tokenize(x) if word not in stop_words])

# 构建词典和语料库
dictionary = corpora.Dictionary(df["tokens"])
corpus = [dictionary.doc2bow(text) for text in df["tokens"]]

# 构建 LDA 模型
lda_model = gensim.models.LdaModel(corpus=corpus,
                                   id2word=dictionary,
                                   num_topics=5,  # 你可以调这个值
                                   passes=10,
                                   random_state=42)

# 输出主题关键词
for i, topic in lda_model.print_topics():
    print(f"Topic {i}: {topic}")

[nltk_data] Downloading package punkt to /Users/zoeyzhou/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zoeyzhou/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Topic 0: 0.060*"x" + 0.014*"function" + 0.014*"n" + 0.013*"b" + 0.010*"k" + 0.010*"f" + 0.009*"fx" + 0.008*"point" + 0.008*"equation" + 0.006*"get"
Topic 1: 0.037*"r" + 0.012*"questions" + 0.010*"outcomes" + 0.009*"use" + 0.009*"even" + 0.008*"calculation" + 0.007*"chatgpt" + 0.007*"question" + 0.006*"lower" + 0.006*"wolframalpha"
Topic 2: 0.017*"number" + 0.014*"numbers" + 0.011*"one" + 0.008*"two" + 0.007*"would" + 0.007*"like" + 0.007*"way" + 0.006*"set" + 0.006*"pi" + 0.006*"fraction"
Topic 3: 0.018*"math" + 0.009*"algebra" + 0.009*"dont" + 0.009*"like" + 0.008*"get" + 0.008*"youre" + 0.007*"time" + 0.007*"good" + 0.006*"know" + 0.006*"understand"
Topic 4: 0.015*"paper" + 0.012*"u" + 0.008*"chance" + 0.008*"use" + 0.006*"youtube" + 0.006*"probability" + 0.005*"practice" + 0.004*"find" + 0.004*"round" + 0.003*"try"


In [12]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# 1. 读取你的CSV数据
df = pd.read_csv("reddit_comments.csv")

# 2. 文本清洗（可简单处理为小写）
df["cleaned_comment"] = df["comment"].astype(str).str.lower()

# 3. 设定你的目标描述（你要找哪些评论跟这个相似）
target_description = "I want to find comments about people’s feelings and perceptions towards math, like whether they enjoy it, fear it, or find it useful."

# 4. 使用 Transformer 模型进行嵌入
model = SentenceTransformer('all-MiniLM-L6-v2')
target_embedding = model.encode(target_description, convert_to_tensor=True)
comment_embeddings = model.encode(df["cleaned_comment"].tolist(), convert_to_tensor=True)

# 5. 计算相似度
cosine_scores = util.cos_sim(target_embedding, comment_embeddings)[0]

# 6. 将相似度加回到 DataFrame
df["similarity"] = cosine_scores.cpu().numpy()

# 7. 按相似度排序，选出最相关的前1000条
top_comments = df.sort_values(by="similarity", ascending=False).head(1000)

# 8. 保存到新CSV文件
top_comments.to_csv("math_perception_comments_top1000.csv", index=False)

In [13]:
import pandas as pd
from bertopic import BERTopic
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import nltk

# 下载 NLTK 所需资源（只需一次）
nltk.download('punkt')
nltk.download('stopwords')

# 读取清洗后的评论数据
top_comments = pd.read_csv("math_perception_comments_top1000.csv")

# 获取英文停用词表和词干器
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()

# 定义预处理函数
def preprocess(text):
    words = word_tokenize(str(text).lower())  # 小写并分词
    words = [stemmer.stem(w) for w in words if w.isalpha() and w not in stop_words]
    return ' '.join(words)

# 应用预处理到所有评论
texts = top_comments["cleaned_comment"].astype(str).apply(preprocess).tolist()

# 主题建模
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(texts)

# 加入主题编号到 DataFrame 中
top_comments["topic"] = topics

# 获取主题信息
topic_info = topic_model.get_topic_info()
print(topic_info.head())

# 保存带主题的数据
top_comments.to_csv("math_perception_with_topics.csv", index=False)

# 可视化主题分布图（展示前5个主题）
topic_model.visualize_barchart(top_n_topics=5).show()


[nltk_data] Downloading package punkt to /Users/zoeyzhou/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zoeyzhou/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


   Topic  Count                           Name  \
0      0    974         0_math_http_learn_like   
1      1     26  1_question_llm_chatgpt_calcul   

                                      Representation  \
0  [math, http, learn, like, algebra, get, proble...   
1  [question, llm, chatgpt, calcul, even, mathema...   

                                 Representative_Docs  
0  [want refresh mathemat use khan academi cours ...  
1  [chatgpt larg languag model design calcul http...  
